<a href="https://colab.research.google.com/github/Condemor-bit/Large-Language-Models-/blob/main/RAG_zephyr_7b_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

========= INSTRUCTIONS ==============

    1º Change the runtime environment to "T4 GPU".
    2º Install the dependencies.
    3º Load the PDF file.
    4º Load the model.
    5º Explain the context of the file.
    
    6º Ask a question.

    TIP: Be patient

=====================================

This notebok was made by Álvaro Morcuende (08 Nov 2023)

In [1]:
#@title 2º)  Install dependencies
%%capture
!pip install accelerate==0.24.1
!pip install transformers==4.35.0
!pip install pypdf==3.17.0
!pip install chromadb==0.4.15
!pip install langchain==0.0.331
!pip install sentence-transformers==2.2.2

In [2]:
#@title 3º) Load PDF file


from google.colab import files
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


uploaded_files = files.upload()
clave = next(iter(uploaded_files))
#valor = uploaded_files[primer_clave]
#print("Clave:", clave)

#read pdf
loader = PyPDFLoader(clave)
docs = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.split_documents(docs)
_text = []
for i in range(len(texts)):
  _text.append(texts[i].page_content)

#embeding and db

embeddings = HuggingFaceEmbeddings()
embedded_tests = embeddings.embed_documents(_text)
db = Chroma.from_documents(texts, embeddings)

Saving aspirina.pdf to aspirina.pdf


In [3]:
#@title 4º) Load the model
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
#@title 5º) Explain the context of the file  (for example: context is based on manufacturer's data sheet)
context_explanation = input("Explain the context: ")

Explain the context: context is based on manufacturer's data sheet


In [5]:
#@title 5º) ask a question

query = input("What can I do for you?: ")


query_vector = embeddings.embed_query(query)
similar_doc = db.similarity_search_by_vector(query_vector, k=2)

contextos=[]
for doc in similar_doc:
    contextos.append(doc.page_content)

cleaned_data = []
for item in contextos:
    cleaned_item = item.replace("\n", "").replace("?", "")
    cleaned_data.append(cleaned_item)

prompt_template = [
    {
        "role": "system",
        "content": f"""TL:DR: Use the following pieces of context to answer the question at the end.
if you don't know the answer, just say that you don't know, don't try to make up an answer.
context:{cleaned_data}
Context Explanation: {context_explanation}.""",
    },
    {"role": "user", "content": f"{query}"},
]
prompt = pipe.tokenizer.apply_chat_template(prompt_template, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
#print(outputs[0]["generated_text"])
output = outputs[0]["generated_text"]
#print(output)
respuesta_intermedia = []
respuesta_intermedia.append(output)
cadena_a_dividir = respuesta_intermedia[0]
cadena_a_dividir2 = cadena_a_dividir.split("<|assistant|>")
print(cadena_a_dividir2[-1])

What can I do for you?: ¿Cuáles son las reacciones adversas más frecuentes?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



Las reacciones adversas más frecuentes del ácido acetilsalicílico son:

1. Trastornos de la sangre y sistema linfático:
   - Aumento del riesgo de hemorragia, hemorragias asociadas al procedimiento, hematomas, epistaxis, hemorragias urogenitales, sangrado gingival, hipoprotrombinemia (con dosis altas).

2. Trastornos gastrointestinales:
   - Úlcera gástrica, úlcera duodenal, hemorragia gastrointestinal (melenas, hematemesis), dolor abdominal y gastrointestinal, dispepsia, diarrea, náuseas, vómitos.

Las reacciones adversas más frecuentes aparecen en el rango de frecuencia de 1/100 a 1/10.
